In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os

In [64]:
df= pd. read_csv('data/IoTProcessed_Data.csv')

In [65]:
df.head()

,date,tempreature,humidity,water_level,N,P,K,Fan_actuator_OFF,Fan_actuator_ON,Watering_plant_pump_OFF,Watering_plant_pump_ON,Water_pump_actuator_OFF,Water_pump_actuator_ON
0,2024-02-08 06:10:00,41,63,100,255,255,255,0.0,1.0,1.0,0.0,1.0,0.0
1,2024-02-08 06:15:00,41,59,100,255,255,255,0.0,1.0,1.0,0.0,1.0,0.0
2,2024-02-08 06:20:00,41,62,100,255,255,255,0.0,1.0,1.0,0.0,1.0,0.0
3,2024-02-08 06:05:00,40,60,100,255,255,255,0.0,1.0,1.0,0.0,1.0,0.0
4,2024-02-08 06:00:00,39,61,100,255,255,255,0.0,1.0,1.0,0.0,1.0,0.0


In [67]:
df.drop(['date','Fan_actuator_OFF','Fan_actuator_ON'], axis=1, inplace=True)

In [68]:
df

,tempreature,humidity,water_level,N,P,K,Watering_plant_pump_OFF,Watering_plant_pump_ON,Water_pump_actuator_OFF,Water_pump_actuator_ON
0,41,63,100,255,255,255,1.0,0.0,1.0,0.0
1,41,59,100,255,255,255,1.0,0.0,1.0,0.0
2,41,62,100,255,255,255,1.0,0.0,1.0,0.0
3,40,60,100,255,255,255,1.0,0.0,1.0,0.0
4,39,61,100,255,255,255,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
37917,3,59,100,255,255,255,1.0,0.0,1.0,0.0
37918,3,60,100,255,255,255,1.0,0.0,1.0,0.0
37919,3,60,100,255,255,255,1.0,0.0,1.0,0.0
37920,3,59,100,255,255,255,1.0,0.0,1.0,0.0


In [69]:
df.drop(["Watering_plant_pump_OFF","Water_pump_actuator_OFF"],axis=1,inplace=True)

In [70]:
df["Watering_plant_pump"]=df["Watering_plant_pump_ON"]
df["Water_pump_actuator"]=df["Water_pump_actuator_ON"]
df.drop(["Watering_plant_pump_ON","Water_pump_actuator_ON"],axis=1,inplace=True)

In [71]:
df.head()

,tempreature,humidity,water_level,N,P,K,Watering_plant_pump,Water_pump_actuator
0,41,63,100,255,255,255,0.0,0.0
1,41,59,100,255,255,255,0.0,0.0
2,41,62,100,255,255,255,0.0,0.0
3,40,60,100,255,255,255,0.0,0.0
4,39,61,100,255,255,255,0.0,0.0


In [73]:
Q1=df["humidity"].quantile(0.25)
Q3=df["humidity"].quantile(0.75)
IQR=Q3-Q1
upper_limit= Q3+(1.5*IQR)
lower_limit=Q1-(1.5*IQR)
print("lower_limit",lower_limit)
print("Upper_limit",upper_limit)

lower_limit 29.5
Upper_limit 81.5


In [74]:
! pip install feature_engine

In [75]:
from feature_engine.outliers import Winsorizer
win=Winsorizer(capping_method='iqr',tail="both", fold=1.5)
win.fit_transform(df[["humidity"]])

,humidity
0,63.0
1,59.0
2,62.0
3,60.0
4,61.0
...,...
37917,59.0
37918,60.0
37919,60.0
37920,59.0


In [76]:
df["Watering_plant_pump"]=df["Watering_plant_pump"].astype("int")
df["Water_pump_actuator"]=df["Water_pump_actuator"].astype("int")

In [99]:
x=df.drop(["Water_pump_actuator"],axis=1)
y=df["Water_pump_actuator"]

In [100]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
x_sample, y_sample = SMOTE(random_state=42).fit_resample(x, y)


In [102]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)
x_train_res, y_train_res = SMOTE(random_state=42).fit_resample(x_train, y_train)


LOGISTIC MODEL

In [105]:
from sklearn.linear_model import LogisticRegression
model1=LogisticRegression()
model1.fit(x_train_res, y_train_res)
#predictions
y_pred_sample=model1.predict(x_sample)
y_pred_test=model1.predict(x_test)

In [106]:
from sklearn.metrics import accuracy_score
sample_acc_1=accuracy_score(y_pred_sample,y_sample)
test_acc_1=accuracy_score(y_pred_test,y_test)

In [107]:
from sklearn.model_selection import cross_val_score
print("Sample_accuracy",sample_acc_1)
print("test_accuracy",test_acc_1)
print("cross_val_score",cross_val_score(model1,x_sample,y_sample,cv=5).mean())

Sample_accuracy 0.9855571073338556
test_accuracy 0.985497692814766
cross_val_score 0.9793066006134067


In [108]:
from sklearn.metrics import precision_score,recall_score,f1_score,confusion_matrix
precision=precision_score(y_pred_sample,y_sample)
recall=recall_score(y_pred_sample,y_sample)
f1=f1_score(y_pred_sample,y_sample)


print("precision_score",precision)
print("recall_score",recall)
print("f1_score",f1)

precision_score 0.9847395851074701
recall_score 0.98635231498857
f1_score 0.9855452902873896


In [109]:
print("confusion_matrix",confusion_matrix(y_pred_sample,y_sample))

confusion_matrix [[28957   448]
 [  400 28909]]


DECISION TREE

In [85]:
from sklearn.tree import DecisionTreeClassifier
model2=DecisionTreeClassifier(criterion='gini',max_depth= 1,splitter= 'best')
model2.fit(x_sample,y_sample)

DecisionTreeClassifier(max_depth=1)

In [86]:
#predictions
y_pred_sample_2=model2.predict(x_sample)
y_pred_test_2=model2.predict(x_test)

sample_acc_2=accuracy_score(y_sample,y_pred_sample_2)
test_acc_2=accuracy_score(y_test,y_pred_test_2)
print("sample_accuracy", sample_acc_2)
print("test accuraccy",test_acc_2)
print("cross_val_score",cross_val_score(model2,x_sample,y_sample,cv=5).mean())

sample_accuracy 0.9911775726402562
test accuraccy 0.9914842885123052
cross_val_score 0.9911776112125302


In [88]:
# testing metrics 
precision_2=precision_score(y_pred_sample_2,y_sample)
recall_2=recall_score(y_pred_sample_2,y_sample)
f1_2=f1_score(y_pred_sample_2,y_sample)


print("precision_score",precision_2)
print("recall_score",recall_2)
print("f1_score",f1_2)

precision_score 0.9862383758558436
recall_score 0.996078026628135
f1_score 0.991133780638094


In [89]:
print("confusion_matrix",confusion_matrix(y_pred_sample_2,y_sample))

confusion_matrix [[29243   404]
 [  114 28953]]


In [90]:
#Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV
estimator=DecisionTreeClassifier()
param_grid={"criterion":["gini","entropy"],"max_depth":list(range(1,6)),"splitter":["best","random"]}
grid=GridSearchCV(estimator,param_grid,cv=5)
grid.fit(x_sample,y_sample)
grid.best_params_

{'criterion': 'gini', 'max_depth': 1, 'splitter': 'best'}

RANDOM FOREST

In [91]:
from sklearn.ensemble import RandomForestClassifier
model3=RandomForestClassifier()
model3.fit(x_sample,y_sample)

RandomForestClassifier()

In [92]:
# predicrtions 
y_pred_sample_3=model3.predict(x_sample)
y_pred_test_3=model3.predict(x_test)

In [93]:
from sklearn. metrics import accuracy_score 
sample_accuracy=accuracy_score(y_pred_sample_3,y_sample)
test_accuracy=accuracy_score(y_pred_test_3,y_test)
print("sample_accuracy",sample_accuracy)
print("test_accuracy",test_accuracy)

sample_accuracy 0.9976496235991416
test_accuracy 0.9979562292429532


In [94]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(model3,x_sample,y_sample,cv=5,scoring="accuracy" ).mean())

0.9868176641123447


In [95]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_pred_sample_3,y_sample))

[[29259    40]
 [   98 29317]]


In [97]:
model3.predict([[45,41,0,148,255,255,1]])

array([1])

In [110]:
import joblib, os
best_model = model3 
os.makedirs("models", exist_ok=True)
joblib.dump(best_model, "models/best_model.pkl")


['models/best_model.pkl']